# Reduce JWST NIRCam Images

## Using the JWST Pipeline plus tips from CEERS

https://github.com/dancoe/mirage

### JWST Pipeline run including recommendations from CEERS program

Micaela Bagley's CEERS notebook ceers_nircam_reduction.ipynb offers detailed help and instructions. Custom reduction scripts and files are also provided:

https://ceers.github.io/releases.html#sdr1

### JWST Pipeline documentation

https://jwst-docs.stsci.edu/jwst-data-reduction-pipeline

https://jwst-docs.stsci.edu/jwst-data-reduction-pipeline/algorithm-documentation/stages-of-processing

https://jwst-pipeline.readthedocs.io/en/latest/jwst/introduction.html

https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/

https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_image3.html

Associations to process and combine multiple images:

https://jwst-pipeline.readthedocs.io/en/latest/jwst/associations/asn_from_list.html

https://jwst-pipeline.readthedocs.io/en/latest/jwst/associations/level3_asn_technical.html

STScI training notebook:

https://github.com/spacetelescope/nircam_calib/blob/master/nircam_calib/training_notebooks/jwst_pipeline_walkthrough.ipynb

## Outstanding issues

* Stars too small (MIRAGE)
* Darks skipped (seem to be inconsistent between MIRAGE and JWST pipeline)
* Alignment between short and long wavelength images (JWST pipeline)

In [ ]:
import jwst
from jwst.pipeline import calwebb_detector1, calwebb_image2, calwebb_image3  # Detector1Pipeline, Image2Pipeline

from jwst.associations.lib.rules_level2_base import DMSLevel2bBase
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base
from jwst.associations import asn_from_list

jwst.__version__

In [ ]:
# CEERS custom pipeline steps
custom_gain_file = 'inputs/gains_v2.1.0/jwst_nircam_gain_nrca1.fits'  # detector1 jump and ramp_fit steps
from applyflat import apply_custom_flat   # remove artifact in simulated F277W, F356W, F444W images
from remstriping import measure_striping  # remove vertical and horizontal striping
# image2
from jwst.skymatch import SkyMatchStep    # run skymatch on each image individually
# image3

In [ ]:
from astropy.io import fits
from glob import glob
import numpy as np
import yaml
import os

In [ ]:
os.environ["CRDS_DATA"] = "$HOME/crds_cache"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"
os.environ["CRDS_CONTEXT"] = 'jwst_0674.pmap'  # CEERS

# Inputs

In [ ]:
filt_to_process = 'F277W'
output_dir = os.path.join('images', filt_to_process)

In [ ]:
lam = int(filt_to_process[1:4])
channel = ['sw', 'lw'][lam > 235]
detector_pixel_scale = {'sw':0.031, 'lw':0.063}[channel]
filt_to_process, lam, channel, detector_pixel_scale

In [ ]:
# Uncalibrated images (generated by MIRAGE)
uncal_files = glob(os.path.join(output_dir, '*_uncal.fits'))
uncal_files = list(np.sort(uncal_files))
len(uncal_files), uncal_files

# Start Processing and Calibrating Images

In [ ]:
# detector1
for uncal_file in uncal_files:
    rate_file = uncal_file.replace('_uncal.fits', '_rate.fits')
    if not os.path.exists(rate_file):
        # 3 minutes w/o dark and w/o superbias
        detector1 = calwebb_detector1.Detector1Pipeline()
        detector1.output_dir = output_dir
        # (if using dark, 3 minutes to download the 3GB dark file -- it seems to do this every time it uses it!)
        detector1.dark_current.skip = True  # Causes errors, 3GB files take forever, and nearly zero anyway, right?
        detector1.superbias.skip = True  # Superbias already subtracted by MIRAGE linearized dark file?
        detector1.ipc.skip = False  # Correct for interpixel capicitance simulated by MIRAGE
        detector1.persistence.skip = True  # Persistence not simulated by MIRAGE
        # Override the gain reference file used for the jump and ramp_fit steps
        detector1.jump.override_gain = custom_gain_file
        detector1.ramp_fit.override_gain = detector1.jump.override_gain
        detector1.save_results = True
        detector1_output = detector1.run(uncal_file)
        
# _uncal.fits -> _rate.fits

In [ ]:
# CEERS Custom Step - remstriping.py
# measure and remove the horizontal and vertical striping from the two countrate images
for uncal_file in uncal_files:
    # 20 seconds
    rate_file = uncal_file.replace('_uncal.fits', '_rate.fits')
    rate_orig_file = rate_file.replace('_rate.fits', '_rate_orig.fits')
    if not os.path.exists(rate_orig_file):
        measure_striping(rate_file, apply_flat=True, mask_sources=True, seedim_directory=output_dir, threshold=0.01)
        # There will be some warnings related to empty slices in the images, where the rows and columns 
        # of reference pixels along the image edges have been masked out of the median calculation.
    
# OLD _rate.fits -> _rate_orig.fits
# NEW _rate.fits

In [ ]:
# image2
for uncal_file in uncal_files:
    rate_file = uncal_file.replace('_uncal.fits', '_rate.fits')
    cal_file  = uncal_file.replace('_uncal.fits', '_cal.fits')
    if not os.path.exists(cal_file):
        # 30 seconds (mostly flat fielding)
        image2 = calwebb_image2.Image2Pipeline()  # Create an instance of the pipeline class
        image2.output_dir = output_dir
        image2.save_results = True
        image2.resample.skip = True  # Don't produce quick-look individual rectified *_i2d.fits
        image2.run(rate_file)  # rate -> cal
        
# _rate.fits -> _cal.fits

In [ ]:
# CEERS Custom Step: Remove A5 Detector Feature in simulated images produced with flipped reference file
# CEERS only produced these files for F277W, F356W, F444W

#if channel == 'lw':
if filt_to_process in 'F277W F356W F444W'.split():
    for uncal_file in uncal_files:
        cal_file    = uncal_file.replace('_uncal.fits', '_cal.fits')
        unflat_file = uncal_file.replace('_uncal.fits', '_unflat.fits')
        if not os.path.exists(unflat_file):
            apply_custom_flat(cal_file)
            # very quick

# OLD _cal.fits -> _unflat.fits
# NEW _cal.fits

In [ ]:
# CEERS Custom Step: Sky Subtraction on each image individually
for uncal_file in uncal_files:
    cal_file  = uncal_file.replace('_uncal.fits', '_cal.fits')
    skymatch_file = uncal_file.replace('_uncal.fits', '_skymatchstep.fits')
    
    if not os.path.exists(skymatch_file):
        file_root_name = os.path.basename(cal_file).split('.')[0]
        association_file = file_root_name + '.json'
        association = asn_from_list.asn_from_list([cal_file], product_name=file_root_name,
                                                  rule=DMS_Level3_Base, asn_rule='Asn_Image')

        with open(association_file, 'w') as fh:
           fh.write(association.dump()[1])
        
        skymatch = SkyMatchStep()
        skymatch.save_results = True
        skymatch.output_dir = output_dir
        skymatch.output_file = file_root_name

        # sky statistics parameters
        skymatch.skymethod = 'local' # the default is global+match, doesn't matter as we're processing files individually
        skymatch.lsigma = 2.0
        skymatch.usigma = 2.0
        skymatch.nclip = 10
        skymatch.upper = 1.0

        skymatch.subtract = True  # subtract calculated sky value from the image (off by default)
        sky = skymatch.run(association_file)  # cal -> skymatchstep
        # jw01433010001_01101_00001_nrca1_skymatchstep.fits
        
# _cal.fits -> _skymatchstep.fits

# Drizzle combine images

In [ ]:
output_file_root_name = 'MACS0647_' + filt_to_process
output_file_root_name

In [ ]:
#association_file = 'MACS0647_%s_image_associations.json' % filt_to_process
association_file = output_file_root_name + '_image_associations.json'
association_file

In [ ]:
#cal_images = [uncal_file.replace('_uncal.fits', '_cal.fits') for uncal_file in uncal_files]
cal_images = [uncal_file.replace('_uncal.fits', '_skymatchstep.fits') for uncal_file in uncal_files]
cal_images

In [ ]:
association = asn_from_list.asn_from_list(cal_images, product_name=output_file_root_name,
                                          rule=DMS_Level3_Base, asn_rule='Asn_Image')

with open(association_file, 'w') as fh:
   fh.write(association.dump()[1])

In [ ]:
if 0:
    # extract reference WCS from image produced previously 
    infile = glob('images/F150W/original/*_i2d.fits')[0]
    hdu = fits.open(infile)
    output_shape = hdu[1].header['NAXIS1'], hdu[1].header['NAXIS2']
    crval = hdu[1].header['CRVAL1'], hdu[1].header['CRVAL2']
    crpix = hdu[1].header['CRPIX1'], hdu[1].header['CRPIX2']
    rotation = np.arccos(hdu[1].header['PC2_2']) / np.pi * 180  # degrees

    print(infile)
    print(output_shape, 'image pixels')
    print(crval, 'CRVAL (RA, Dec of CRPIX)')
    print(crpix, 'CRPIX (reference pixel)')
    print(rotation, 'rotation (degrees)')

    # next we'll generate the new image matched to this WCS

In [ ]:
# 10 minutes for 16 SW images -> 1 image
#  3 minutes for  4 SW images -> 1 image

image3 = calwebb_image3.Image3Pipeline()  # calwebb_image3.py

image3.tweakreg.skip = True  # skip TweakReg since simulated images are perfectly aligned
image3.skymatch.skip = True  # skip SkyMatch since already did it above for each image individually

output_pixel_scale = 0.03  # (arcsec) for both SW and LW
image3.resample.pixel_scale = output_pixel_scale
#image3.resample.pixel_scale = 0.04  # (arcsec) for both SW and LW
#image3.resample.pixel_scale_ratio = output_pixel_scale / detector_pixel_scale

if 0:  # Output images WCS aligned to existing images
    # Without rotation input, drizzled LW image is rotated slightly 0.484 deg clockwise wrt SW image
    image3.resample.output_shape = output_shape
    image3.resample.crval = crval
    image3.resample.crpix = crpix
    image3.resample.rotation = rotation

image3.output_dir = output_dir
image3.save_results = True  # _i2d.fits

#image3.outlier_detection.skip = False
#image3.source_catalog.snr_threshold = 5  # 20
#image3.source_catalog.output_file = "MACS0647_%s_cat.ecsv" % filt_to_process
image3.source_catalog.output_file = output_file_root_name + '_cat.ecsv'

image3.run(association_file)

# _i2d.fits [1] SCI, [2] ERR, [3] CONtext, [4] WHT, [5] VAR_POISSON, [6] VAR_RNOISE, [7] VAR_FLAT

In [ ]:
# Did it produce the output image the way we asked? Almost. CRPIX is a bit off

infile = glob('images/%s/*_i2d.fits' % filt_to_process)[0]
hdu = fits.open(infile)
output_shape = hdu[1].header['NAXIS1'], hdu[1].header['NAXIS2']
crval = hdu[1].header['CRVAL1'], hdu[1].header['CRVAL2']
crpix = hdu[1].header['CRPIX1'], hdu[1].header['CRPIX2']
rotation = np.arccos(hdu[1].header['PC2_2']) / np.pi * 180  # degrees

print(infile)
print(output_shape, 'image pixels')
print(crval, 'CRVAL (RA, Dec of CRPIX)')
print(crpix, 'CRPIX (reference pixel)')
print(rotation, 'rotation (degrees)')